### Installation


In [3]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu
!pip install pyPDF2

### Import Python Modules

In [5]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
import PyPDF2
warnings.filterwarnings("ignore")

### Importing OpenAI API Key

In [6]:
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

### Extracting Tables from PDF

In [7]:
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [10]:
pdf_text = extract_text_from_pdf("/content/financial-wellness-journal-english.pdf")

In [12]:
print(len(pdf_text))

80030


In [56]:
chunk_size = 1000
chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]

In [57]:
print(chunks)

['Financial Wellness Journal \nIntroduction\nGetting started Pre-assessmentTABLE OF CONTENTS\nChapter 1: All About Financial Wellness\nChapter \nintroductionHow FinEd can \nhelp you achieve \nfinancial wellnessThe life stages\nof money Facing these \ndifficulties head onActivity: Visioning \nexercise\nChapter 2: All About Saving\nCase study: Mang \nRafael’s saving \nproblem Common \nmisconceptions \nabout savingWhat are the goals \nof saving?What are the tools \nwe can use to start \nsaving?Where do we save?\nCase study: Aling \nYolanda insures her \ndaughter’s future What is insurance? Why do we need \nto insure ourselves \nand our family?What are the \ndifferent kinds\nof insurance?Chapter wrap-up\nChapter 4: All About CreditChapter 3: All About Insurance\nChapter 5: All About Investing\nConclusion3 4\n6 7 8 9 11\n12 13 14 15 20\n23 21 24\n27 28 29 29 31\n33 34 35 36\n38 39 39 4037\n41 42 43 43 46Where should our \nsavings go?What are the \nmethods of \nbudgeting?Chapter wrap-up\nCas

In [58]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in chunks]

In [59]:
embeddings_dim = len(embeddings[0])

In [60]:
embeddings_np = np.array(embeddings).astype('float32')

In [61]:
index = faiss.IndexFlatL2(embeddings_dim)

In [62]:
index.add(embeddings_np)

In [83]:
user_message = "How to be a good investor?"

In [84]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')
_, indices = index.search(query_embedding_np, 5)

In [85]:
retrieved_docs = [chunks[i] for i in indices[0]]
retrieved_docs

['NVESTING\n53CHAPTER WRAP-UP!\nASK!\nInvesting can be exciting. \nThis is where you can actually \nmake your money work for you \nand reap large benefits later. \nHowever, be sure to only invest \nin products and with businesses \nyou fully understand. Do your \nresearch. Ask any and all \nquestions you can think of,\neven if you feel dumb asking \nthem. Remember that you \nare possibly risking your \nhard-earned cash. Legitimate \ninvestment businesses \nunderstand that people have\nto be cautious about their \nmoney and will gladly answer \nyour questions. BE CURIOUS!\nIt will also be wise to do your \nown investigation. Check out \n(legitimate!) internet sources \non the type of investment you \nare interested in. Ask a friend or \nfamily member you trust who \nis knowledgeable about the \ntopic. Familiarize yourself with \ninvestment terms and lingo so \nthat you won’t be overwhelmed \nor confused once you’re deep \ninto considering where and \nwith whom to invest. Ask banks \nand

In [86]:
context = ' '.join(retrieved_docs)

In [87]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [88]:
system_prompt = "You are a helpful assistant."

In [89]:
struct = [{"role": "system", "content": system_prompt}]

In [90]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [91]:
print(response)

To be a good investor, consider the following key principles:

1. **Educate Yourself**: Understand the basics of investing, including different types of investment products (stocks, bonds, mutual funds, etc.), how the market works, and the risks involved. Familiarize yourself with investment terminology to avoid confusion.

2. **Know Your Investment Personality**: Assess your risk appetite. Are you aggressive, conservative, or somewhere in between? This will guide your investment choices. For instance, aggressive investors might opt for equity funds, while conservative investors might prefer money market funds.

3. **Do Your Research**: Before investing, thoroughly research the investment products and businesses you are considering. Look for legitimate sources of information and consult with knowledgeable friends or family members.

4. **Ask Questions**: Don’t hesitate to ask questions about anything you don’t understand. A legitimate investment company will be happy to provide you wit